# TP 3 - Word2Vec & FastText

L'objectif de ce TP est de mettre en œuvre l'algorithme de word2vec (CBOW et Skip-gram) et d'explorer FastText.

## Exercice 1 – Entrainer son propre word2vec

In [ ]:
# 1. Importer les dépendances
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import gensim
from gensim.models import Word2Vec

warnings.filterwarnings(action='ignore')

nltk.download('punkt')

# 2. Importer les données et déclarer quelques variables
# Lecture du fichier Alice in Wonderland
try:
    sample = open("../data/alice_wonderland.txt", "r")
    s = sample.read()
except FileNotFoundError:
    print("Fichier non trouvé. Vérifiez le chemin.")
    s = ""

In [ ]:
# 3. Prétraitement des données
# Remplacer les sauts de ligne par des espaces
f = s.replace("\n", " ")

data = []
# Iterer sur chaque phrase
for i in sent_tokenize(f):
    temp = []
    # Tokeniser la phrase en mots
    for j in word_tokenize(i):
        temp.append(j.lower())
    data.append(temp)

print(f"Nombre de phrases: {len(data)}")
print(f"Premier exemple: {data[0]}")

In [ ]:
# 4. Entrainement du CBOW
# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5)

# Print results
print("Cosine similarity between 'alice' " + "and 'wonderland' - CBOW : ", model1.wv.similarity('alice', 'wonderland'))
print("Cosine similarity between 'alice' " + "and 'machines' - CBOW : ", model1.wv.similarity('alice', 'machines') if 'machines' in model1.wv else "'machines' not in vocab")

In [ ]:
# 5. Entrainement du Skip-gram
# Create Skip Gram model (sg=1)
model2 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5, sg = 1)

# Print results
print("Cosine similarity between 'alice' " + "and 'wonderland' - Skip Gram : ", model2.wv.similarity('alice', 'wonderland'))
print("Cosine similarity between 'alice' " + "and 'machines' - Skip Gram : ", model2.wv.similarity('alice', 'machines') if 'machines' in model2.wv else "'machines' not in vocab")

### Question :
Jouez avec le paramètre vector_size =[2, 10, 500] sur le Skipgram et CBOW, quel est l’effet sur les distances ?

## Exercice 2 – Utiliser un modèle pré-entrainé
*Note: Le téléchargement du modèle peut prendre du temps.*

In [ ]:
import gensim.downloader as api

# Télécharger le modèle pré entrainé (Google News 300)
# Attention : c'est un gros fichier (~1.5 Go)
try:
    wv = api.load('word2vec-google-news-300')
    print("Modèle chargé !")
except Exception as e:
    print("Erreur lors du chargement ou téléchargement trop long:", e)

In [ ]:
# Trouver la similitude entre 2 mots
if 'wv' in locals():
    pairs = [
        ('car', 'minivan'),
        ('car', 'bicycle'),
        ('car', 'airplane'),
        ('car', 'cereal'),
        ('car', 'communism'),
    ]
    for w1, w2 in pairs:
        print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))
      
    # Jouez un peu avec la similitude des mots
    print(wv.most_similar(positive=['car', 'minivan'], topn=5))
    print(wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car']))

## Exercice 3 - FastText

Utilisation de FastText sur le corpus Brown.

In [ ]:
from gensim.models.fasttext import FastText
nltk.download('brown')
from nltk.corpus import brown

# Chargement du corpus Brown
brown_tokens = [brown.words(fileids=f) for f in brown.fileids()]

print("Training FastText model...")
# Instantiation et entrainement
ft_model = FastText(vector_size=100, window=5, min_count=5, sentences=brown_tokens, epochs=10)
print("Training complete.")

In [ ]:
# Similitudes
print("Similaire 'nation':", ft_model.wv.most_similar('nation'))
# FastText gère bien les mots inconnus (OOV) grâce aux n-grams de caractères
print("Similaire 'accomodation' (typo):", ft_model.wv.most_similar('accomodation'))

In [ ]:
# Visualisation avec PCA
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def plot_pca(model, words):
    # Récupérer les vecteurs
    vectors = [model.wv[w] for w in words if w in model.wv]
    if not vectors:
        return
        
    pca = PCA(n_components=2)
    result = pca.fit_transform(vectors)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(result[:, 0], result[:, 1])
    
    for i, word in enumerate(words):
        plt.annotate(word, xy=(result[i, 0], result[i, 1]))
    plt.show()

words_to_plot = ['king', 'queen', 'man', 'woman', 'car', 'bicycle', 'bus']
plot_pca(ft_model, words_to_plot)